<a href="https://colab.research.google.com/github/NNLogic/NNPosterPublic/blob/main/NNPoster%E3%80%80Json%E7%94%BB%E5%83%8F%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#説明
PDF/PNG/JPGからNNPoster用のJSON画像ファイルを作成するスクリプトになります。

Betaテスト版です。

★上から順番に実行してください。

Googleドライブからファイルをコピーする場合、[2-A]を実行後、png/jpgは[2-A1]、pdfは[2-A2]を実行してください。

ローカルコンピューターからPDFをアップロードする場合、[2-B]を実行してください。

※NNPosterに同梱されているUnityのスクリプトでは、crunch圧縮によりファイルサイズを小さくできます。

※スライドを変換する場合、出力画像サイズは1024を推奨しています。



In [ ]:
#@title [1] 出力画像の横サイズ
max_width = '1024' #@param [1024, 2048, 4096] {allow-input: true}

Googleドライブに保存されたフォルダからpngを読み込みます。

In [ ]:
#@title [2-A] Googleドライブをマウントする

!cd /content
%cd  /content

import ipywidgets as widgets
from google.colab import drive

if 'drive' in locals():
  drive.flush_and_unmount()

drive.mount('/content/drive')

/content
Mounted at /content/drive


In [ ]:
from pkgutil import extend_path
#@title [2-A1] Googleドライブのフォルダからpng, jpgをコピーする
!cd /content
%cd  /content

!rm -f *.pdf *.png *.json

import os
import shutil

folder_path = "/content/drive/MyDrive" #@param {type:"string"}
rename = "on" #@param ["on", "off"]
#@markdown ※ファイル名の末尾の数字をサムネイル位置にする場合、rename=offにしてください。
copy_dest = "/content"  # コピー先のディレクトリパスを指定してください

os.chdir(folder_path)

image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]

no = 0

for file in image_files:
    no = no + 1
    src_path = os.path.join(folder_path, file)
    dest_path = os.path.join(copy_dest, file)
    shutil.copy(src_path, dest_path)
    if rename == "on":
      file_name, file_extension = os.path.splitext(dest_path)
      new_name = "image-"+str(no)+file_extension
      os.rename(dest_path, os.path.join(os.path.dirname(dest_path), new_name))


/content


In [ ]:
#@title [2-A2] GoogleドライブからPDFをコピーする
!cd /content
%cd  /content

!rm -f *.pdf *.png *.json

import os
import shutil

pdf_path = "/content/drive/MyDrive/NNPoster (3).pdf" #@param {type:"string"}
copy_dest = "/content"  # コピー先のディレクトリパスを指定してください

file_name = os.path.basename(pdf_path)

dest_path = os.path.join(copy_dest, file_name)
shutil.copy(pdf_path, dest_path)

!apt-get install -y poppler-utils
!pdftoppm -scale-to $width -png "$file_name" image

/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
#@title [2-B] コンピューターに保存されたPDFをアップロードする
!cd /content
%cd  /content

!rm -f *.pdf *.png *.json

from google.colab import files
import os

result = files.upload()
pdf_file_name = list(result.keys())[0]
base_file_name = os.path.splitext(pdf_file_name)[0]

!apt-get install -y poppler-utils

!pdftoppm -scale-to $max_width -png "$pdf_file_name" image

/content


Saving NNPoster (3).pdf to NNPoster (3).pdf
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
#@title [3]JSONファイルへの変換
!cd /content
%cd  /content

!sudo apt-get install imagemagick
!pip install wand

import os, sys
from wand.image import Image
import datetime
import base64
import json
import re


no = 0
images = []

files = os.listdir("./")
sorted_files = sorted(files)
for file_name in sorted_files:
    if file_name.endswith(".png") or file_name.endswith(".jpg"):

      im = Image(filename=file_name)

      if im.width > max_width:
        target_width = int(max_width)
        original_width = im.width
        original_height = im.height

        # アスペクト比を維持して変換後の高さを計算
        target_height = int(original_height * (target_width / original_width))

        # 幅と高さを指定してリサイズ
        im.resize(target_width, target_height)

      im.flip()

      im.convert("RGBA")
      im.options['dds:mipmaps'] = "0"
      im.compression = "dxt5"
      im.format = 'dds'
      binary_data = im.make_blob()
      b64_txt = base64.b64encode(binary_data[128:])

      type_str = "10"  # type DXT1 9 DXT5 10

      if im.compression == "dxt1":
        type_str = "9"
      match = re.search(r"\d+$", file_name)
      if match:
        no = int(match.group())
      else:
        no = no + 1
      if no > 200
        no = 200
      if no < 1
        no = 1
      image_dict = {"no":str(no),"type":type_str,"width":str(im.width),"height":str( im.height),"base64":b64_txt.decode('utf-8')};
      images.append(image_dict)

dict= {"version":"001","images":images}


# output
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d_%H%M%S")
json_file_name = formatted_datetime + '.json'
with open(json_file_name, "w") as outputFile:
  json.dump(dict, outputFile, indent=2 )


/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
#@title [4]ファイルのダウンロード
from google.colab import files

files.download(json_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>